In [2]:
!pip install benzinga

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.6/127.6 kB 3.8 MB/s eta 0:00:00
  Created wheel for benzinga: filename=benzinga-1.21-py3-none-any.whl size=12231 sha256=de0aeb3a23dcc7e406bf1422a56f9b14aaf5ffcfee707d3782077dfd2f57b4ed
  Stored in directory: /root/.cache/pip/wheels/54/90/98/31c6b9e221854508d3ffa6ebf3d155bcd49c211d3f52341b84
Successfully built benzinga
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.15
    Uninstalling urllib3-1.26.15:
      Successfully uninstalled urllib3-1.26.15


In [7]:
from benzinga import financial_data
from benzinga import news_data

import pandas as pd
import numpy as np
import json
import inspect
import datetime

In [8]:
api_key = ""
news = news_data.News(api_key)

In [ ]:
# Optional:
# pagesize (int) - default is 15
# page (int) - default is 0
# display_output (str) - select from (full, abstract, headline)
# base_date (str) - "YYYY-MM-DD" The date to query for calendar data. Shorthand for date_from and date_to if they are the same. Defaults for latest.
# date_from (str) - "YYYY-MM-DD"
# date_to (str) - "YYYY-MM-DD"
# last_id (str) - The last ID to start paging from and sorted by and sorted by the last updated date.
# updated_since (str) - he last updated unix timestamp (UTC) to pull and sort by.
# publish_since (str) - The last publish unix timestamp (UTC) to pull and sort by.
# company_tickers (str)
# channel (str) - multiple channels separated by comma.

In [5]:
# The maximum output of the API call is 100 stories in one call.
# While loop that takes acount of from and to date, and loop until it gets all the dates we need
# Concat it into a new dataframe, return the dataframe

def benzinga_call(ticker, fromdate, todate):
    stories = news.news(display_output = 'full', company_tickers = ticker, pagesize = 100, date_from = fromdate, date_to = todate)
    df = pd.DataFrame(stories)
    df['created'] = pd.to_datetime(df['created']).dt.date
    fromdate = (df.iloc[-1,2] - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    
    one_month_be4_todate = (datetime.datetime.strptime(todate, '%Y-%m-%d') - datetime.timedelta(days=30)).strftime('%Y-%m-%d')
    while fromdate < one_month_be4_todate:
        stories = news.news(display_output = 'full', company_tickers = ticker, pagesize = 100, date_from = fromdate, date_to = todate)
        stories_df = pd.DataFrame(stories)
        stories_df['created'] = pd.to_datetime(stories_df['created']).dt.date
        df = pd.concat([df,stories_df]).drop_duplicates(subset=['id']).reset_index(drop=True)
        fromdate = (df.iloc[-1,2] - datetime.timedelta(days=1)).strftime('%Y-%m-%d')
    return df

In [6]:
# Assign values to the function's parameters
ticker = 'AAPL'
fromdate = "2020-02-18"
todate = "2023-02-18"
df = benzinga_call(ticker, fromdate, todate)

2023-05-01 06:38:59 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2020-02-18&dateTo=2023-02-18&tickers=AAPL
2023-05-01 06:39:00 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2020-03-14&dateTo=2023-02-18&tickers=AAPL
2023-05-01 06:39:00 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2020-05-04&dateTo=2023-02-18&tickers=AAPL
2023-05-01 06:39:01 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb78f70d1&pageSize=100&displayOutput=full&dateFrom=2020-06-22&dateTo=2023-02-18&tickers=AAPL
2023-05-01 06:39:02 [info     ] Status Code: 200 Endpoint: http://api.benzinga.com/api/v2/news/?token=318da1f2bee64e3caf1c22dfb7

In [ ]:
df

In [ ]:
# output the dataframe
df.to_csv('AAPL_benzinga_2020_to_2023.csv', index=False)